# Scrape Diamonds
Find web request URL that site is using to get data. This is found in network tab of developers tools. Only 20 rows at a time are possible. The `rowStart` steps 20 each iteration.

In [8]:
import pandas as pd
import requests

url = 'http://www.diamondse.info/webService.php?shape=none&minCarat=0.2&maxCarat=30&minColor=1'\
      '&maxColor=9&minPrice=100&maxPrice=100000&minCut=5&maxCut=1&minClarity=1&maxClarity=10'\
      '&minDepth=0&maxDepth=90&minWidth=0&maxWidth=90&gia=1&ags=1&egl=0&oth=0&currency=USD'\
      '&rowStart={}&sortCol=price&sortDir=DESC'

dfs = []
start = 0
step = 20
pages = 10000
stop = step * pages
for row in range(start, stop, step):
    if row % 1000 == 0:
        print(row)
    r = requests.get(url.format(row))
    table = f"<table>{r.text}</table>"
    dfs.extend(pd.read_html(table))

In [5]:
# clean up

dia = pd.concat(dfs, ignore_index=True)
dia2 = dia.iloc[:, 1:11]
dia2.columns = ['shape', 'carat', 'cut', 'color', 'clarity', 'table', 'depth', 'cert', 'xyz', 'price']
dia2['price'] = dia2['price'].str.replace(r'[$,]', '').astype('int')
dia2[['x', 'y', 'z']] = dia2['xyz'].str.extract('([0-9.]*) x ([0-9.]*) x ([0-9.]*)').astype('float')
dia3 = dia2.drop(columns='xyz')
dia3.head()

,shape,carat,cut,color,clarity,table,depth,cert,price,x,y,z
0,Round,5.01,Ideal,F,SI1,57.0,62.7,GIA,99996,10.81,10.88,6.80
1,Oval,4.01,Good,F,VS2,55.0,68.8,GIA,99989,12.62,8.23,5.66
2,Round,4.25,Ideal,E,SI1,57.0,62.1,GIA,99940,10.38,10.33,6.43
3,Round,5.20,Ideal,J,VS2,58.0,61.6,GIA,99890,11.12,11.08,6.84
4,Round,3.63,Ideal,F,VVS2,60.0,59.1,GIA,99848,10.08,10.12,5.97


In [6]:
dia3.shape

(179180, 12)

In [7]:
dia3.to_csv('../data/diamonds.csv', index=False)